<a href="https://colab.research.google.com/github/GustavoFelips/DadosPagueBem/blob/main/Indices_PagueBem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importação da base

In [3]:
import pandas as pd

#Criando o DataFrame através do CSV
df = pd.read_csv('data_pague_bem.csv')

#Retirando a Coluna Index que veio no CSV
df = df.drop(df.columns[0],axis=1)

#Verificando o DataFrame
df.head(3)

,codigo,identificador,nº parcela,nº documento,vencimento,pagamento,tempo para pagar,valor conta,valor pago,diferença,lead
0,1,34680,9,4728,2021-09-10,2021-09-11,-1,599.0,530.0,-69.0,3
1,1,34680,4,4728,2021-04-10,2021-04-13,-3,599.0,530.0,-69.0,3
2,1,34680,3,4728,2021-03-10,2021-03-11,-1,599.0,530.0,-69.0,3


# Índice De Pagamento

##Base

In [4]:
import pandas as pd

#Criando o DataFrame através do CSV
df = pd.read_csv('data_pague_bem.csv')

#Retirando a Coluna Index que veio no CSV
df = df.drop(df.columns[0],axis=1)

#Verificando o DataFrame
df.head(3)



,codigo,identificador,nº parcela,nº documento,vencimento,pagamento,tempo para pagar,valor conta,valor pago,diferença,lead
0,1,34680,9,4728,2021-09-10,2021-09-11,-1,599.0,530.0,-69.0,3
1,1,34680,4,4728,2021-04-10,2021-04-13,-3,599.0,530.0,-69.0,3
2,1,34680,3,4728,2021-03-10,2021-03-11,-1,599.0,530.0,-69.0,3


##Função Nota

In [13]:
#Função que calcula a nota do índice a partir da média do tempo de pagamento
def calcular_indice_pagamento(media):
  if media < -15:
    return 0
  elif -15 <= media < -10:
    return 2.5
  elif -10 <= media < -5:
    return 5.0
  elif -5 <= media < -2:
    return 7.0
  elif -2 <= media < 0:
    return 8
  elif media >= 0:
    return 10

##Índice de todos os clientes



In [14]:

#Tira as médias do tempo de pagamento por cliente e retorna suas devidas notas
def tabela_indice_Pagamento(df):
  if 'identificador' in df.columns and 'tempo para pagar' in df.columns:

    #Calcula a média de tempo de pagamento por cliente
    medias_por_cliente = df.groupby('identificador')['tempo para pagar'].mean().reset_index()

    #Calcula para cada média uma nota
    medias_por_cliente['i.pag'] = medias_por_cliente['tempo para pagar'].apply(calcular_indice_pagamento)

    #Modifica nome da coluna
    medias_por_cliente.rename(columns={'tempo para pagar' : 'media_tempo_pagamento'},inplace=True)

    return medias_por_cliente

  else:

    print("As colunas 'identificador' ou 'tempo para pagar' não foram encontradas no arquivo CSV.")

#Chamada da função
tabela_indice_Pagamento(df)



,identificador,media_tempo_pagamento,i.pag
0,16075,-9.333333,5.0
1,16077,6.666667,10.0
2,16078,0.333333,10.0
3,16080,2.222222,10.0
4,16081,-117.888889,0.0
...,...,...,...
1102,40438,0.888889,10.0
1103,42085,6.555556,10.0
1104,42228,-7.777778,5.0
1105,43881,4.000000,10.0


##Índice por Cliente



In [15]:
def indice_pagamento_por_cliente(cliente_id):
    linhas_cliente = df[df['identificador']==cliente_id]
    media_tempo_pagamento = linhas_cliente['tempo para pagar'].mean()
    return {'media_tempo_pagamento':media_tempo_pagamento,'indice_pagamento':calcular_indice_pagamento(media_tempo_pagamento)}

#Exemplo
indice_pagamento_por_cliente(16075)


{'media_tempo_pagamento': -9.333333333333334, 'indice_pagamento': 5.0}

# Índice de Regularidade

##Base

In [16]:
import pandas as pd

#Criando o DataFrame através do CSV
df = pd.read_csv('data_pague_bem.csv')

#Retirando a Coluna Index que veio no CSV
df = df.drop(df.columns[0],axis=1)

#Verificando o DataFrame
df.head(3)

,codigo,identificador,nº parcela,nº documento,vencimento,pagamento,tempo para pagar,valor conta,valor pago,diferença,lead
0,1,34680,9,4728,2021-09-10,2021-09-11,-1,599.0,530.0,-69.0,3
1,1,34680,4,4728,2021-04-10,2021-04-13,-3,599.0,530.0,-69.0,3
2,1,34680,3,4728,2021-03-10,2021-03-11,-1,599.0,530.0,-69.0,3


##Funçao Índice Regularidade

In [17]:
def calcular_indice_regularidade(desvio_padrao):
    # Atribuir a nota com base no desvio padrão que já foi calculado
    if desvio_padrao > 30:
        return 0
    elif 20 < desvio_padrao <= 30:
        return 2.5
    elif 10 < desvio_padrao <= 20:
        return 5
    elif 5 < desvio_padrao <= 10:
        return 7
    elif 3 < desvio_padrao <= 5:
        return 8
    elif 0 <= desvio_padrao <= 3:
        return 10


##Índice de todos os clientes

In [24]:
def tabela_indice_regularidade (df) :
    df_indice = df.groupby('identificador')['tempo para pagar'].agg(
    desvio_padrao='std',  # Calcula o desvio padrão do tempo de pagamento por cliente
    quantidade_pagamentos='count').reset_index() # Quantidade de pagamentos por cliente (pode ser útil)

    # Aplicar a função calcular_indice_regularidade
    df_indice['i.reg'] = df_indice['desvio_padrao'].apply(calcular_indice_regularidade)

    return df_indice

# Exibir as primeiras linhas do DataFrame
tabela_indice_regularidade(df)

,identificador,desvio_padrao,quantidade_pagamentos,i.reg
0,16075,24.909837,9,2.5
1,16077,10.037430,9,5.0
2,16078,2.828427,9,10.0
3,16080,2.048034,9,10.0
4,16081,83.257799,9,0.0
...,...,...,...,...
1102,40438,2.803767,9,10.0
1103,42085,4.362084,9,8.0
1104,42228,11.278051,9,5.0
1105,43881,23.113849,9,2.5


##Índice por Cliente

In [20]:
def indice_regularidade_por_cliente(cliente_id):
    # Filtrar o DataFrame original pelo cliente_id
    df_cliente = df[df['identificador'] == cliente_id]

    # Calcular o desvio padrão do 'tempo para pagar'
    desvio_padrao = df_cliente['tempo para pagar'].std()

    # Chamar a função que calcula o índice de regularidade
    return calcular_indice_regularidade(desvio_padrao)

# Exemplo de uso
indice_regularidade_por_cliente(16075)


2.5

#Tabela Completa

In [30]:
df_completo = pd.merge(tabela_indice_Pagamento(df), tabela_indice_regularidade(df), on='identificador', how='outer')
df_completo = df_completo[['identificador','quantidade_pagamentos','media_tempo_pagamento','desvio_padrao','i.pag','i.reg']]
df_completo.head(3)

,identificador,quantidade_pagamentos,media_tempo_pagamento,desvio_padrao,i.pag,i.reg
0,16075,9,-9.333333,24.909837,5.0,2.5
1,16077,9,6.666667,10.037430,10.0,5.0
2,16078,9,0.333333,2.828427,10.0,10.0
